In [139]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statsmodels.api as sm
from scipy.special import gamma, loggamma, factorial
import scipy.stats
from scipy.interpolate import interp1d
from matplotlib.ticker import AutoMinorLocator 
from matplotlib import rc, font_manager
from matplotlib.lines import Line2D
from matplotlib import colors as mcolors
from mycolorpy import colorlist as mcp
from matplotlib import legend_handler
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
import collections
import matplotlib
import shapely.geometry
import shapely.ops 
from geopy.geocoders import Nominatim
import geopandas as gpd
import mapclassify
import jenkspy
import contextily as cx
import os
import rioxarray as rx
import pyreadr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
# import plots
import seaborn as sns
from prophet import Prophet
import datetime
import warnings
warnings.filterwarnings("ignore")

In [140]:
wd = os.path.dirname(os.getcwd())

In [141]:
lookup = pd.read_csv("./data/geodemographics/lookup-2011.csv")

In [142]:
lookup_manchester = lookup[lookup['LAD11NM'].isin(['Bolton', 'Bury', 'Manchester', 'Oldham', 'Rochdale', 
                                                   'Salford', 'Stockport', 'Tameside', 'Trafford', 'Wigan'])].reset_index(drop=True)


In [143]:
lookup_manchester.columns

Index(['OA11CD', 'LSOA11CD', 'LSOA11NM', 'MSOA11CD', 'MSOA11NM', 'LAD11CD',
       'LAD11NM', 'LAD11NMW', 'ObjectId'],
      dtype='object')

In [144]:
lookup_london = lookup[lookup['LAD11NM'].isin(['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent',
                                              'Bromley', 'Camden', 'City of London', 'Croydon', 'Ealing',
                                              'Enfield', 'Greenwich', 'Hackney', 'Hammersmith and Fulham',
                                              'Haringey', 'Harrow', 'Havering', 'Hillingdon', 'Hounslow',
                                              'Islington', 'Kensington and Chelsea', 'Kingston upon Thames',
                                              'Lambeth', 'Lewisham', 'Merton', 'Newham', 'Redbridge',
                                              'Richmond upon Thames', 'Southwark', 'Sutton', 'Tower Hamlets',
                                              'Waltham Forest', 'Wandsworth', 'Westminster'])].reset_index(drop=True)

In [145]:
landcover = pd.read_csv("./data/geodemographics/lsoa_land_cover.csv", index_col=0)

In [146]:
landcover_manchester = landcover[landcover['LSOA11CD'].isin(lookup_manchester['LSOA11CD'])].reset_index(drop=True)


In [147]:
landcover_london = landcover[landcover['LSOA11CD'].isin(lookup_london['LSOA11CD'])].reset_index(drop=True)


In [148]:
sf_lsoas = gpd.read_file('./data/geodemographics/LSOA_Dec_2011_Boundaries.gpkg')

In [149]:
sf_lsoas_manchester = sf_lsoas[sf_lsoas['LSOA11CD'].isin(lookup_manchester['LSOA11CD'])].reset_index(drop=True)


In [150]:
sf_lsoas_london = sf_lsoas[sf_lsoas['LSOA11CD'].isin(lookup_london['LSOA11CD'])].reset_index(drop=True)


In [151]:
landcover_manchester_geometry = landcover_manchester.merge(sf_lsoas_manchester, on='LSOA11CD')
landcover_manchester_geometry = gpd.GeoDataFrame(landcover_manchester_geometry)

landcover_manchester_geometry = landcover_manchester_geometry.drop(['LSOA11NM_y'], axis=1)
landcover_manchester_geometry = landcover_manchester_geometry.rename(columns={'LSOA11NM_x': 'LSOA11NM'})

remove_columns = []
for x in landcover_manchester_geometry:
    if x[-5:-1] == '2000' or x[-5:-1] == '2006' or x[-5:-1] == '2018':  
        remove_columns.append(x)
        
landcover_manchester_geometry = landcover_manchester_geometry.drop(remove_columns, axis=1)

landcover_manchester_geometry.to_file('./data/geodemographics/manchester_land_cover_2011.gpkg')


In [152]:
landcover_london_geometry = landcover_london.merge(sf_lsoas_london, on='LSOA11CD')
landcover_london_geometry = gpd.GeoDataFrame(landcover_london_geometry)

landcover_london_geometry = landcover_london_geometry.drop(['LSOA11NM_y'], axis=1)
landcover_london_geometry = landcover_london_geometry.rename(columns={'LSOA11NM_x': 'LSOA11NM'})

remove_columns = []
for x in landcover_london_geometry:
    if x[-5:-1] == '2000' or x[-5:-1] == '2006' or x[-5:-1] == '2018':  
        remove_columns.append(x)
        
landcover_london_geometry = landcover_london_geometry.drop(remove_columns, axis=1)

landcover_london_geometry.to_file('./data/geodemographics/london_land_cover_2011.gpkg')
